In [1]:
from getpass import getpass
import pandas as pd
from mysql.connector import connect, Error
import warnings
import requests
import json
import codecs
import time
import re
import datetime

In [2]:
#ignore warnings
warnings.filterwarnings("ignore")

# connet to server

In [3]:
#connect to sever
try:
    conn = connect(
        host="localhost",
        user="root", #input("Enter username: ")
        password="guoxiaochen1997", #getpass("Enter password: ")
        database="Masks") #after first run, needs to be left blank on first run to create DB
    cursor = conn.cursor()
    print(conn)
except Error as e:
    print(e)

# create tables

In [4]:
#create tables
SQL_DB = "Masks"

In [5]:
#create Amazon price history table with transformed time
def main():
    
    SQL_TABLE = "datedprices_amazon"
    SQL_TABLE_DEF = "(" + \
            "asin CHAR(10) CHARACTER SET ascii NOT NULL" + \
            ",time BIGINT(15) NOT NULL" + \
            ",PRIMARY KEY (asin, time)" + \
            ",price BIGINT(20)" + \
            ")"

    create_sql_table(SQL_TABLE, SQL_TABLE_DEF)
   
def create_sql_table(SQL_TABLE, SQL_TABLE_DEF):
    try:
        query = "CREATE DATABASE IF NOT EXISTS " + SQL_DB
        print(query)
        cursor.execute(query);
        
        query = "CREATE TABLE IF NOT EXISTS " + SQL_DB + "." + SQL_TABLE + " " + SQL_TABLE_DEF + ";";
        print(query)
        cursor.execute(query)
        return
    except IOError as e:
        print(e)

if __name__ == "__main__":
    main()

CREATE DATABASE IF NOT EXISTS Masks
CREATE TABLE IF NOT EXISTS Masks.datedprices_amazon (asin CHAR(10) CHARACTER SET ascii NOT NULL,time BIGINT(15) NOT NULL,PRIMARY KEY (asin, time),price BIGINT(20));


In [6]:
#create Marketplace New price history table with transformed time
def main():
    
    SQL_TABLE = "datedprices_new"
    SQL_TABLE_DEF = "(" + \
            "asin CHAR(10) CHARACTER SET ascii NOT NULL" + \
            ",time BIGINT(15) NOT NULL" + \
            ",PRIMARY KEY (asin, time)" + \
            ",price BIGINT(20)" + \
            ")"

    create_sql_table(SQL_TABLE, SQL_TABLE_DEF)
   
def create_sql_table(SQL_TABLE, SQL_TABLE_DEF):
    try:
        query = "CREATE DATABASE IF NOT EXISTS " + SQL_DB
        print(query)
        cursor.execute(query);
        
        query = "CREATE TABLE IF NOT EXISTS " + SQL_DB + "." + SQL_TABLE + " " + SQL_TABLE_DEF + ";";
        print(query)
        cursor.execute(query)
        return
    except IOError as e:
        print(e)

if __name__ == "__main__":
    main()

CREATE DATABASE IF NOT EXISTS Masks
CREATE TABLE IF NOT EXISTS Masks.datedprices_new (asin CHAR(10) CHARACTER SET ascii NOT NULL,time BIGINT(15) NOT NULL,PRIMARY KEY (asin, time),price BIGINT(20));


In [7]:
#create Marketplace Used price history table with transformed time
def main():
    
    SQL_TABLE = "datedprices_used"
    SQL_TABLE_DEF = "(" + \
            "asin CHAR(10) CHARACTER SET ascii NOT NULL" + \
            ",time BIGINT(15) NOT NULL" + \
            ",PRIMARY KEY (asin, time)" + \
            ",price BIGINT(20)" + \
            ")"

    create_sql_table(SQL_TABLE, SQL_TABLE_DEF)
   
def create_sql_table(SQL_TABLE, SQL_TABLE_DEF):
    try:
        query = "CREATE DATABASE IF NOT EXISTS " + SQL_DB
        print(query)
        cursor.execute(query);
        
        query = "CREATE TABLE IF NOT EXISTS " + SQL_DB + "." + SQL_TABLE + " " + SQL_TABLE_DEF + ";";
        print(query)
        cursor.execute(query)
        return
    except IOError as e:
        print(e)

if __name__ == "__main__":
    main()

CREATE DATABASE IF NOT EXISTS Masks
CREATE TABLE IF NOT EXISTS Masks.datedprices_used (asin CHAR(10) CHARACTER SET ascii NOT NULL,time BIGINT(15) NOT NULL,PRIMARY KEY (asin, time),price BIGINT(20));


In [8]:
#create Sales Rank history table with transformed time
def main():
    
    SQL_TABLE = "datedsales_ranks"
    SQL_TABLE_DEF = "(" + \
            "asin CHAR(10) CHARACTER SET ascii NOT NULL" + \
            ",category BIGINT(20) NOT NULL" + \
            ",time BIGINT(15) NOT NULL" + \
            ",PRIMARY KEY (asin, category, time)" + \
            ",sales_rank BIGINT(10) DEFAULT NULL" + \
            ")"
    
    create_sql_table(SQL_TABLE, SQL_TABLE_DEF)
   
def create_sql_table(SQL_TABLE, SQL_TABLE_DEF):
    try:
        query = "CREATE DATABASE IF NOT EXISTS " + SQL_DB
        print(query)
        cursor.execute(query);
        
        query = "CREATE TABLE IF NOT EXISTS " + SQL_DB + "." + SQL_TABLE + " " + SQL_TABLE_DEF + ";";
        print(query)
        cursor.execute(query)
        return
    except IOError as e:
        print(e)

if __name__ == "__main__":
    main()

CREATE DATABASE IF NOT EXISTS Masks
CREATE TABLE IF NOT EXISTS Masks.datedsales_ranks (asin CHAR(10) CHARACTER SET ascii NOT NULL,category BIGINT(20) NOT NULL,time BIGINT(15) NOT NULL,PRIMARY KEY (asin, category, time),sales_rank BIGINT(10) DEFAULT NULL);


# extract data from exitsting tables

In [9]:
def convert(timestamp,method):
    utctime = datetime.datetime.utcfromtimestamp(timestamp/1000)
    utctime_week = utctime.isocalendar()[1]
    date_year_month = str(utctime.year)+f"{utctime.month:02}"#year-month
    date_year_month_day = date_year_month+f"{utctime.day:02}"#year-month-day
    date_year_month_day_hour = date_year_month_day+f"{utctime.hour:02}" #year-month-day_hour
    if utctime.year == utctime.isocalendar()[0] and utctime_week == 53:
        date_year_week = str(utctime.year)+'52'
    elif utctime.year != utctime.isocalendar()[0] and utctime_week == 53:
        date_year_week = str(utctime.year)+'01'
    else: date_year_week = str(utctime.year)+f"{utctime_week:02}"#year-week
    switcher={
        0:int(date_year_month),
        1:int(date_year_month_day),
        2:int(date_year_month_day_hour),
        3:int(date_year_week)}
    
    return switcher.get(method)

In [10]:
def price_datedprice(SQL_TABLE1,SQL_TABLE2):
    try:
        #get amazon price history from amazon_price
        sql = "SELECT asin,price,time from " + SQL_TABLE1
        data = pd.read_sql(sql,conn)
        #transform the time into date as year-month-day
        data["time"] = data["time"].apply(convert, method = 3)#year-week
    
        #change the name of column to match the column name of new table
        data.columns = ['asin','price','time']
    
        # creating column list for insertion
        cols = "`,`".join([str(i) for i in data.columns.tolist()])
        
        # Insert DataFrame recrds one by one.
        for i,row in data.iterrows():
            sql = "INSERT IGNORE INTO " +SQL_TABLE2+ "(`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s);"
            cursor.execute(sql, tuple(row))
        conn.commit()
        
    except IOError as e:
        print(e) 

In [11]:
price_datedprice("prices_amazon","datedprices_amazon")

In [12]:
price_datedprice("prices_new","datedprices_new")

In [13]:
price_datedprice("prices_used","datedprices_used")

In [14]:
#salesrank
def salesrank_datedsalesrank(SQL_TABLE1,SQL_TABLE2):
    try:
        #get amazon price history from amazon_price
        sql = "SELECT asin,sales_rank,category,time from " + SQL_TABLE1
        data = pd.read_sql(sql,conn)
        #transform the time into date as year-month-day
        data["time"] = data["time"].apply(convert, method = 3)#year-week
    
        #change the name of column to match the column name of new table
        data.columns = ['asin','sales_rank','category','time']
    
        # creating column list for insertion
        cols = "`,`".join([str(i) for i in data.columns.tolist()])
        
        # Insert DataFrame recrds one by one.
        for i,row in data.iterrows():
            sql = "INSERT IGNORE INTO " +SQL_TABLE2+ "(`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s);"
            cursor.execute(sql, tuple(row))
        conn.commit()
        
    except IOError as e:
        print(e) 

In [15]:
salesrank_datedsalesrank("sales_ranks","datedsales_ranks")